# YOLOv5 xView Training

## Imports

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 15637, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 15637 (delta 123), reused 173 (delta 101), pack-reused 15393
Receiving objects: 100% (15637/15637), 14.65 MiB | 6.27 MiB/s, done.
Resolving deltas: 100% (10647/10647), done.


In [ ]:
!pip install sahi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 16.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=151835fdd665c4369e50ecc952d5ab8a6b65c130d9ea1c03b1c2fbea5619c69a
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire
  Attempting uninstall: click
    Found existing installation: click 8.1.3
    Uninstalling click-8.1.3:
      Successfully uninstalled click-8.1.3


In [ ]:
!apt-get install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 43.0 kB of archives.
After this operation, 115 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tree amd64 1.8.0-1 [43.0 kB]
Fetched 43.0 kB in 1s (42.5 kB/s)
Selecting previously unselected package tree.
(Reading database ... 122518 files and directories currently installed.)
Preparing to unpack .../tree_1.8.0-1_amd64.deb ...
Unpacking tree (1.8.0-1) ...
Setting up tree (1.8.0-1) ...
Processing triggers for man-db (2.9.1-1) ...


In [ ]:
# arrange an instance segmentation model for test
from sahi.utils.mmdet import (
    download_mmdet_retinanet_model,
    download_mmdet_cascade_mask_rcnn_model,
    download_mmdet_config,
)

# import required functions, classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image

import fire
from sahi.scripts.slice_coco import slice
from tqdm import tqdm

import shutil

In [ ]:
import random
from collections import defaultdict
from pathlib import Path
from typing import Dict, List

import fire
import numpy as np
from PIL import Image
from sahi.utils.coco import Coco, CocoAnnotation, CocoCategory, CocoImage
from sahi.utils.file import load_json, save_json
from tqdm import tqdm
import matplotlib.pyplot as plt

COLAB = True

Mounted at /content/drive


In [ ]:
import os
#from mmdet.datasets import build_dataset
#from mmdet.models import build_detector
#from mmdet.apis import train_detector
import glob
import cv2
import shutil
import random
import os.path as osp
import json
import re
import xml.etree.ElementTree as ET
from typing import Dict, List

In [ ]:
from sahi.utils.coco import Coco
from sahi.utils.file import Path, increment_path

## Global Variables

In [ ]:
ROOT_DIR_PATH = os.path.abspath('..')

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

  ROOT_DIR_PATH = os.path.abspath('drive/MyDrive/Spatial_Finance_Transport/')

XVIEW_CLASS_LABELS_PATH = "/content/drive/MyDrive/Spatial_Finance_Transport/MMDetCV/xview_class_labels.txt"
XVIEW_JSON_CATEGORY_MAPPING_PATH = "/content/drive/MyDrive/Spatial_Finance_Transport/MMDetCV/category_id_mapping.json"

if COLAB:
  XVIEW_TRAIN_IMAGES_DIR = "/content/xView/train_images"
  XVIEW_TRAIN_GEOJSON_PATH = "/content/xView/xView_train.geojson"
  XVIEW_CLASS_LABELS_PATH = "/content/drive/MyDrive/Spatial_Finance_Transport/MMDetCV/xview_class_labels.txt"
  XVIEW_JSON_CATEGORY_MAPPING_PATH = "/content/drive/MyDrive/Spatial_Finance_Transport/MMDetCV/category_id_mapping.json"
  XVIEW_OUTPUT_DIR = "/content/xView"

In [ ]:
SLICE_SIZE_LIST = [512]
OVERLAP_RATIO_LIST = [0.25]
IGNORE_NEGATIVE_SAMPLES = False

XVIEW_IMAGES_DIR = "/content/xView/train_images"
XVIEW_COCO_TRAIN_GEOJSON_PATH = '/content/xView/train.json'
XVIEW_COCO_VAL_GEOJSON_PATH = "/content/xView/val.json"
XVIEW_SLICE_TRAIN_OUTPUT_DIR = '/content/xView_slice_train'
XVIEW_SLICE_VAL_OUTPUT_DIR = '/content/xView_slice_val'

## Helper Functions

In [ ]:
def coco_to_yolov5(
    image_dir: str,
    dataset_json_path: str,
    train_split: str = 1,
    project: str = "runs/coco2yolov5",
    name: str = "exp",
    seed: str = 1,
    disable_symlink=True,
):
    """
    Args:
        images_dir (str): directory for coco images
        dataset_json_path (str): file path for the coco json file to be converted
        train_split (str): set the training split ratio
        project (str): save results to project/name
        name (str): save results to project/name"
        seed (int): fix the seed for reproducibility
        disable_symlink (bool): required in google colab env
    """

    # increment run
    save_dir = Path(increment_path(Path(project) / name, exist_ok=False))
    # load coco dict
    coco = Coco.from_coco_dict_or_path(
        coco_dict_or_path=dataset_json_path,
        image_dir=image_dir,
    )
    # export as yolov5
    coco.export_as_yolov5(
        output_dir=str(save_dir),
        train_split_rate=train_split,
        numpy_seed=seed,
        disable_symlink=disable_symlink,
    )

    print(f"COCO to YOLOv5 conversion results are successfully exported to {save_dir}")

In [ ]:
def copy_file(source_file, target_dir):
    """
    Copies a file from source file path to target directory path.

    Args:
        source_file (str): Path to the source file.
        target_dir (str): Path to the target directory.

    Returns:
        str: Path to the copied file in the target directory.
    """
    # Extract the filename from the source file path
    filename = os.path.basename(source_file)
    # Construct the target file path by joining the target directory and filename
    target_file = os.path.join(target_dir, filename)
    # Copy the file from source to target
    shutil.copy2(source_file, target_file)
    return target_file

In [ ]:
def move_files(source_dir, destination_dir):
    """
    Move all files from the source directory to the destination directory.

    Args:
        source_dir (str): Path to the source directory.
        destination_dir (str): Path to the destination directory.

    Returns:
        None
    """
    # Check if source directory exists
    if not os.path.exists(source_dir):
        print(f"Source directory '{source_dir}' does not exist.")
        return

    # Check if destination directory exists, if not create it
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Get all files in the source directory
    files = os.listdir(source_dir)

    # Loop through all files and move them to the destination directory
    for file in files:
        source_file = os.path.join(source_dir, file)
        destination_file = os.path.join(destination_dir, file)
        shutil.move(source_file, destination_file)
        print(f"Moved file '{file}' to '{destination_dir}'.")

    print("All files moved successfully.")

In [ ]:
def get_labels(fname):
    """
    Gets label data from a geojson label file
    Args:
        fname: file path to an xView geojson label file
    Output:
        Returns three arrays: coords, chips, and classes corresponding to the
            coordinates, file-names, and classes for each ground truth.
    Modified from https://github.com/DIUx-xView.
    """
    data = load_json(fname)

    coords = np.zeros((len(data["features"]), 4))
    chips = np.zeros((len(data["features"])), dtype="object")
    classes = np.zeros((len(data["features"])))
    image_name_to_annotation_ind = defaultdict(list)

    for i in tqdm(range(len(data["features"])), "Parsing xView data"):
        if data["features"][i]["properties"]["bounds_imcoords"] != []:
            b_id = data["features"][i]["properties"]["image_id"]
            # https://github.com/DIUx-xView/xView1_baseline/issues/3
            if b_id == "1395.tif":
                continue
            val = np.array(
                [
                    int(num)
                    for num in data["features"][i]["properties"][
                        "bounds_imcoords"
                    ].split(",")
                ]
            )
            chips[i] = b_id
            classes[i] = data["features"][i]["properties"]["type_id"]

            image_name_to_annotation_ind[b_id].append(i)

            if val.shape[0] != 4:
                print("Issues at %d!" % i)
            else:
                coords[i] = val
        else:
            chips[i] = "None"

    return coords, chips, classes, image_name_to_annotation_ind

In [ ]:
def get_ordered_image_name_list(image_name_to_annotation_ind: Dict):
    image_name_list: List[str] = list(image_name_to_annotation_ind.keys())

    def get_image_ind(image_name: str):
        return int(image_name.split(".")[0])

    image_name_list.sort(key=get_image_ind)

    return image_name_list

In [ ]:
def xview_to_coco(
    train_images_dir,
    train_geojson_path,
    output_dir,
    train_split_rate=0.9,
    category_id_remapping=None,
):
    """
    Converts visdrone-det annotations into coco annotation.
    Args:
        train_images_dir: str
            'train_images' folder directory
        train_geojson_path: str
            'xView_train.geojson' file path
        output_dir: str
            Output folder directory
        train_split_rate: bool
            Train split ratio
        category_id_remapping: dict
            Used for selecting desired category ids and mapping them.
            If not provided, xView mapping will be used.
            format: str(id) to str(id)
    """

    # init vars
    category_id_to_name = {}
    with open(XVIEW_CLASS_LABELS_PATH, encoding="utf8") as f:
        lines = f.readlines()
    for line in lines:
        category_id = line.split(":")[0]
        category_name = line.split(":")[1].replace("\n", "")
        category_id_to_name[category_id] = category_name

    if category_id_remapping is None:
        category_id_remapping = load_json(XVIEW_JSON_CATEGORY_MAPPING_PATH)
    category_id_remapping

    # init coco object
    coco = Coco()
    # append categories
    for category_id, category_name in category_id_to_name.items():
        if category_id in category_id_remapping.keys():
            remapped_category_id = category_id_remapping[category_id]
            coco.add_category(
                CocoCategory(id=int(remapped_category_id), name=category_name)
            )

    # parse xview data
    coords, chips, classes, image_name_to_annotation_ind = get_labels(
        train_geojson_path
    )
    image_name_list = get_ordered_image_name_list(image_name_to_annotation_ind)

    # convert xView data to COCO format
    for image_name in tqdm(image_name_list, "Converting xView data into COCO format"):
        # create coco image object
        width, height = Image.open(Path(train_images_dir) / image_name).size
        coco_image = CocoImage(file_name=image_name, height=height, width=width)

        annotation_ind_list = image_name_to_annotation_ind[image_name]

        # iterate over image annotations
        for annotation_ind in annotation_ind_list:
            bbox = coords[annotation_ind].tolist()
            category_id = str(int(classes[annotation_ind].item()))
            coco_bbox = [bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]]
            if category_id in category_id_remapping.keys():
                category_name = category_id_to_name[category_id]
                remapped_category_id = category_id_remapping[category_id]
            else:
                continue
            # create coco annotation and append it to coco image
            coco_annotation = CocoAnnotation(
                bbox=coco_bbox,
                category_id=int(remapped_category_id),
                category_name=category_name,
            )
            if coco_annotation.area > 0:
                coco_image.add_annotation(coco_annotation)
        coco.add_image(coco_image)

    result = coco.split_coco_as_train_val(train_split_rate=train_split_rate)

    train_json_path = Path(output_dir) / "train.json"
    val_json_path = Path(output_dir) / "val.json"
    save_json(data=result["train_coco"].json, save_path=train_json_path)
    save_json(data=result["val_coco"].json, save_path=val_json_path)

In [ ]:
def merge_jsons(dir):
  result = []
  for f in glob.glob("*.json"):
      with open(f, "rb") as infile:
          result.append(json.load(infile))

  with open("merged_file.json", "wb") as outfile:
      json.dump(result, outfile)

In [ ]:
def slice_xview(image_dir: str, dataset_json_path: str, output_dir: str):
    total_run = len(SLICE_SIZE_LIST) * len(OVERLAP_RATIO_LIST)
    current_run = 1
    for slice_size in SLICE_SIZE_LIST:
        for overlap_ratio in OVERLAP_RATIO_LIST:
            tqdm.write(
                f"{current_run} of {total_run}: slicing for slice_size={slice_size}, overlap_ratio={overlap_ratio}"
            )
            slice(
                image_dir=image_dir,
                dataset_json_path=dataset_json_path,
                output_dir=output_dir,
                slice_size=slice_size,
                overlap_ratio=overlap_ratio,
            )
            current_run += 1

## xView Dataset

### tar files

In [ ]:
%cd ~/../content/
!wget --header="Host: d307kc0mrhucc3.cloudfront.net" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-US,en;q=0.9,ar;q=0.8,zh-CN;q=0.7,zh;q=0.6" --header="Referer: https://challenge.xviewdataset.org/" "https://d307kc0mrhucc3.cloudfront.net/train_images.tgz?Expires=1683470535&Signature=LWID~2nzpA-JTHAcXWKcIZKVFToiFcMmQsT8pzhhamV~l6g1mDsSxdn~hQTez7smRbK1m2dzmqB8Cv73-e0pIQXEVV1JEmCICTq2HfoFXGKDS82cCob5IRGjrvpzH1zpRMC4AEzp4t6AYKxB0Htf~p9dP1sInvbhBJ00dNklbKv4Ln1ssPYGz2ZUNGrbrYNW1zSJlG5kbLxOMMTjhtTnLv7B7CbjGsHP6vqn1Dd~vixYuqi~C88v8NHdJqmFJjIcdzmOFfsnOIgxWCMBkBO1HHV-wc0~HuBcc--DQfcGTHTDnfp22Jsm2ipxITdxZw4sX1jEbt4D7kYuogoHkdtS0w__&Key-Pair-Id=APKAIKGDJB5C3XUL2DXQ" -c -O 'train_images.tgz'
!wget --header="Host: d307kc0mrhucc3.cloudfront.net" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-US,en;q=0.9,ar;q=0.8,zh-CN;q=0.7,zh;q=0.6" --header="Referer: https://challenge.xviewdataset.org/" "https://d307kc0mrhucc3.cloudfront.net/train_labels.tgz?Expires=1683470535&Signature=Z6fyXghTRn09l4jH5LJZoBY-FeOI-VJUEU1U6LO4RvmpWvxlZUggXm8PFo2Km2qRCdGwXzAzXfrIf46YOGxd8uFJmsbq0hbDaT0A8EK3E9UzrTrqi8qegDZHOQOlSe0Px5kyG1xSLcAE-6RU2VfyM3Vsl8~Yj9KScLFSALwB~VCK8GNu9jdgXp7ov6doaIkevRgp2xbrUm1WaO9Emh4jZqGzp4ShcrSfZP4rgRWl-L7Ulz-Hlq1Q6op9r1T4gxGICBZqr1AqTkrrJCgITVZmNrOlYekelnzKOT7onanMfuGeVuSwtKZpsGRBNbpd6ZbVdmRVWjvvKi~-jONMBPAW3Q__&Key-Pair-Id=APKAIKGDJB5C3XUL2DXQ" -c -O 'train_labels.tgz'
!wget --header="Host: d307kc0mrhucc3.cloudfront.net" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7" --header="Accept-Language: en-US,en;q=0.9,ar;q=0.8,zh-CN;q=0.7,zh;q=0.6" --header="Referer: https://challenge.xviewdataset.org/" "https://d307kc0mrhucc3.cloudfront.net/val_images.tgz?Expires=1683470535&Signature=cJ3lwEQkXIUWYGngU69knM4fA~Gio1nR8mPkz05V-gAU2HnKpHmSAjgUe6e9Lz1abhrArBcCJAe6qNr7y-5dqVgOdEQL0Y3wCiiYUfEgcCHD1M8AMTeKcToc-zYOFpsjTZcNe0VDhOu8R7OyGZsPxtcTA8daRMXpdpA4i6UugDchh2VYSMeQvrOIWtjso0aZny3jKzDNNFmEixrmsT0AJC76kdiu03ada5vzvY0e4kyb7r65K3-SYd1bJSx5O3tW1SKl90f0i6frL3COgc9H~1smR8MUNyxeT55UYbohQpwGtxEfnyVVWEJOuLe7ukN02M3sxoG0FjbW4SYuVn3kXw__&Key-Pair-Id=APKAIKGDJB5C3XUL2DXQ" -c -O 'val_images.tgz'

/content
--2023-05-07 08:44:50--  https://d307kc0mrhucc3.cloudfront.net/train_images.tgz?Expires=1683470535&Signature=LWID~2nzpA-JTHAcXWKcIZKVFToiFcMmQsT8pzhhamV~l6g1mDsSxdn~hQTez7smRbK1m2dzmqB8Cv73-e0pIQXEVV1JEmCICTq2HfoFXGKDS82cCob5IRGjrvpzH1zpRMC4AEzp4t6AYKxB0Htf~p9dP1sInvbhBJ00dNklbKv4Ln1ssPYGz2ZUNGrbrYNW1zSJlG5kbLxOMMTjhtTnLv7B7CbjGsHP6vqn1Dd~vixYuqi~C88v8NHdJqmFJjIcdzmOFfsnOIgxWCMBkBO1HHV-wc0~HuBcc--DQfcGTHTDnfp22Jsm2ipxITdxZw4sX1jEbt4D7kYuogoHkdtS0w__&Key-Pair-Id=APKAIKGDJB5C3XUL2DXQ
Resolving d307kc0mrhucc3.cloudfront.net (d307kc0mrhucc3.cloudfront.net)... 13.227.250.197, 13.227.250.117, 13.227.250.77, ...
Connecting to d307kc0mrhucc3.cloudfront.net (d307kc0mrhucc3.cloudfront.net)|13.227.250.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15413902447 (14G) [application/gzip]
Saving to: ‘train_images.tgz’

train_images.tgz    100%[===================>]  14.35G   101MB/s    in 1m 54s  

2023-05-07 08:46:45 (129 MB/s) - ‘train_images.tgz’ saved [15413

In [ ]:
%mkdir xView/

In [ ]:
!tar -xvzf "train_images.tgz" -C "xView/"
!tar -xvzf "train_labels.tgz" -C "xView/"
!tar -xvzf "val_images.tgz" -C "xView/"     #[run this cell to extract tar.gz files]

./train_images/
./train_images/10.tif
./train_images/._100.tif
./train_images/100.tif
./train_images/._102.tif
./train_images/102.tif
./train_images/1036.tif
./train_images/1037.tif
./train_images/104.tif
./train_images/1042.tif
./train_images/1044.tif
./train_images/1046.tif
./train_images/1048.tif
./train_images/1049.tif
./train_images/105.tif
./train_images/1050.tif
./train_images/1051.tif
./train_images/1052.tif
./train_images/1053.tif
./train_images/1056.tif
./train_images/1058.tif
./train_images/106.tif
./train_images/1061.tif
./train_images/1063.tif
./train_images/1065.tif
./train_images/1067.tif
./train_images/1068.tif
./train_images/107.tif
./train_images/1070.tif
./train_images/1072.tif
./train_images/1076.tif
./train_images/1077.tif
./train_images/1079.tif
./train_images/1080.tif
./train_images/1081.tif
./train_images/1084.tif
./train_images/1085.tif
./train_images/1086.tif
./train_images/1087.tif
./train_images/1088.tif
./train_images/1089.tif
./train_images/._109.tif
./tra

### Move train and val to same dir

In [ ]:
# Example usage
source_dir = "/content/xView/val_images"
destination_dir = "/content/xView/train_images"
move_files(source_dir, destination_dir)

Moved file '981.tif' to '/content/xView/train_images'.
Moved file '2340.tif' to '/content/xView/train_images'.
Moved file '447.tif' to '/content/xView/train_images'.
Moved file '2554.tif' to '/content/xView/train_images'.
Moved file '882.tif' to '/content/xView/train_images'.
Moved file '1122.tif' to '/content/xView/train_images'.
Moved file '359.tif' to '/content/xView/train_images'.
Moved file '412.tif' to '/content/xView/train_images'.
Moved file '35.tif' to '/content/xView/train_images'.
Moved file '1060.tif' to '/content/xView/train_images'.
Moved file '2342.tif' to '/content/xView/train_images'.
Moved file '1102.tif' to '/content/xView/train_images'.
Moved file '827.tif' to '/content/xView/train_images'.
Moved file '539.tif' to '/content/xView/train_images'.
Moved file '1811.tif' to '/content/xView/train_images'.
Moved file '1177.tif' to '/content/xView/train_images'.
Moved file '912.tif' to '/content/xView/train_images'.
Moved file '2563.tif' to '/content/xView/train_images'.
Mo

### Check Tree Structure

In [ ]:
%cd ~/../content/
# Check the directory structure of the data

!tree xView

/content
xView
├── train_images
│   ├── 100.tif
│   ├── 102.tif
│   ├── 1036.tif
│   ├── 1037.tif
│   ├── 1038.tif
│   ├── 1040.tif
│   ├── 1042.tif
│   ├── 1043.tif
│   ├── 1044.tif
│   ├── 1046.tif
│   ├── 1048.tif
│   ├── 1049.tif
│   ├── 104.tif
│   ├── 1050.tif
│   ├── 1051.tif
│   ├── 1052.tif
│   ├── 1053.tif
│   ├── 1054.tif
│   ├── 1056.tif
│   ├── 1058.tif
│   ├── 105.tif
│   ├── 1060.tif
│   ├── 1061.tif
│   ├── 1062.tif
│   ├── 1063.tif
│   ├── 1064.tif
│   ├── 1065.tif
│   ├── 1066.tif
│   ├── 1067.tif
│   ├── 1068.tif
│   ├── 1069.tif
│   ├── 106.tif
│   ├── 1070.tif
│   ├── 1071.tif
│   ├── 1072.tif
│   ├── 1073.tif
│   ├── 1075.tif
│   ├── 1076.tif
│   ├── 1077.tif
│   ├── 1079.tif
│   ├── 107.tif
│   ├── 1080.tif
│   ├── 1081.tif
│   ├── 1082.tif
│   ├── 1084.tif
│   ├── 1085.tif
│   ├── 1086.tif
│   ├── 1087.tif
│   ├── 1088.tif
│   ├── 1089.tif
│   ├── 108.tif
│   ├── 1090.tif
│   ├── 1091.tif
│   ├── 1092.tif
│   ├── 1094.tif
│   ├── 1095.tif
│   ├── 1097.tif
│   ├─

## Convert to COCO

In [ ]:
xview_to_coco(XVIEW_TRAIN_IMAGES_DIR, XVIEW_TRAIN_GEOJSON_PATH, XVIEW_OUTPUT_DIR)

Converting xView data into COCO format: 100%|██████████| 846/846 [01:13<00:00, 11.54it/s]


## xView Slice COCO

In [ ]:
%mkdir /content/xView_slice_train/
%mkdir /content/xView_slice_val

In [ ]:
# train
slice_xview(image_dir=XVIEW_IMAGES_DIR, dataset_json_path=XVIEW_COCO_TRAIN_GEOJSON_PATH, output_dir=XVIEW_SLICE_TRAIN_OUTPUT_DIR)

# val
#slice_xview(image_dir=XVIEW_IMAGES_DIR, dataset_json_path=XVIEW_COCO_VAL_GEOJSON_PATH, output_dir=XVIEW_SLICE_VAL_OUTPUT_DIR)

1 of 1: slicing for slice_size=512, overlap_ratio=0.25
Slicing step is starting...
indexing coco dataset annotations...


100%|██████████| 761/761 [33:08<00:00,  2.61s/it]


Sliced dataset for 'slice_size: 512' is exported to /content/xView_slice_train


## COCO to YOLOv5 

In [ ]:
# train
coco_to_yolov5(image_dir = XVIEW_SLICE_TRAIN_OUTPUT_DIR+'/train_images_512_025', dataset_json_path=XVIEW_SLICE_TRAIN_OUTPUT_DIR+'/train_512_025.json', train_split=0.8)

# val
#coco_to_yolov5(image_dir = XVIEW_SLICE_VAL_OUTPUT_DIR+'/val_images_512_025', dataset_json_path=XVIEW_SLICE_VAL_OUTPUT_DIR+'/val_512_025.json')

indexing coco dataset annotations...


100%|██████████| 10338/10338 [00:34<00:00, 298.25it/s]


COCO to YOLOv5 conversion results are successfully exported to runs/coco2yolov5/exp


In [ ]:
!tree /content/runs

Streaming output truncated to the last 5000 lines.
        │   ├── 2125_0_1536_512_2048.jpg
        │   ├── 2125_0_1536_512_2048.txt
        │   ├── 2125_0_1920_512_2432.jpg
        │   ├── 2125_0_1920_512_2432.txt
        │   ├── 2125_0_2480_512_2992.jpg
        │   ├── 2125_0_2480_512_2992.txt
        │   ├── 2125_0_384_512_896.jpg
        │   ├── 2125_0_384_512_896.txt
        │   ├── 2125_0_768_512_1280.jpg
        │   ├── 2125_0_768_512_1280.txt
        │   ├── 2125_1152_0_1664_512.jpg
        │   ├── 2125_1152_0_1664_512.txt
        │   ├── 2125_1152_1152_1664_1664.jpg
        │   ├── 2125_1152_1152_1664_1664.txt
        │   ├── 2125_1152_2480_1664_2992.jpg
        │   ├── 2125_1152_2480_1664_2992.txt
        │   ├── 2125_1152_384_1664_896.jpg
        │   ├── 2125_1152_384_1664_896.txt
        │   ├── 2125_1152_768_1664_1280.jpg
        │   ├── 2125_1152_768_1664_1280.txt
        │   ├── 2125_1536_0_2048_512.jpg
        │   ├── 2125_1536_0_2048_512.txt
        │   ├── 2125_1536_1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Train

To train only road vehicles:

change yolov5/utils/dataloaders.py line 533 to: 

include_class = [4,5,6,7,8,9,19,11,12,13,14,15,18,19]

For first time running the notebook:
- --data /content/runs/coco2yolov5/exp/data.yml
-- /content/yolov5/runs/train/exp/weights/best.pt

In [ ]:
!python /content/yolov5/train.py --img 512 --batch 2 --epochs 20 --data /content/runs/coco2yolov5/exp/data.yml --weights /content/drive/MyDrive/Spatial_Finance_Transport/models/object_detection_models/best.pt --cache

requirements: YOLOv5 requirement "gitpython" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.3 MB/s eta 0:00:00

requirements: 1 package updated per ['gitpython']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

train: weights=/content/drive/MyDrive/Spatial_Finance_Transport/models/object_detection_models/best.pt, cfg=, data=/content/runs/coco2yolov5/exp/data.yml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=2, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, c

In [ ]:
# save best model weights to drive 
copy_file('/content/yolov5/runs/train/exp/weights/best.pt', '/content/drive/MyDrive/Spatial_Finance_Transport/models/object_detection_models/')

'/content/drive/MyDrive/Spatial_Finance_Transport/models/object_detection_models/best.pt'

## Visualize